In [3]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [4]:
import os
import math

import torch
from torch.nn import functional as F
import pytorch_lightning as pl
from torchvision.datasets import MNIST
from torchmetrics import Accuracy
from torchvision.transforms import ToTensor
from torch import utils
from torch import nn
from torchmore import flex, layers
# from pl_bolts.datamodules.mnist_datamodule import MNISTDataModule


In [9]:

class LightningClassifier(pl.LightningModule):
    def __init__(self, network, config):
        super(LightningClassifier, self).__init__()
        self.network = network
        self.lr = config.get("lr", 0.1)
        self.optim = getattr(torch.optim, config.get("optim", "SGD"))
        self.accuracy = Accuracy()

    def forward(self, x):
        y = self.network(x)
        return torch.log_softmax(y, dim=1)

    def configure_optimizers(self):
        return self.optim(self.network.parameters(), lr=self.lr)

    def training_step(self, train_batch, batch_idx, prefix=""):
        x, y = train_batch
        logits = self.forward(x)
        loss = F.nll_loss(logits, y)
        acc = self.accuracy(logits, y)
        self.log(prefix + "loss", loss)
        self.log(prefix + "acc", acc)
        return loss

    def validation_step(self, batch, batch_idx):
        return self.training_step(batch, batch_idx, prefix="val_")

In [10]:
train_ds = list(MNIST(os.getcwd(), download=True, transform=ToTensor()))
train_loader = utils.data.DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=0)
test_ds = list(MNIST(os.getcwd(), download=True, transform=ToTensor(), train=False))
test_loader = utils.data.DataLoader(test_ds, batch_size=64, shuffle=True, num_workers=0)

In [11]:
network = nn.Sequential(
    nn.Flatten(),
    flex.Linear(1024),
    flex.BatchNorm(),
    nn.ReLU(),
    flex.Linear(10),
)
flex.shape_inference(network, (4, 784))

In [12]:
config={"lr": 0.3}
model = LightningClassifier(network, config)
trainer = pl.Trainer(limit_train_batches=100, max_epochs=20)
trainer.fit(model=model, train_dataloaders=train_loader)
trainer.validate(model=model, dataloaders=test_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/tbreuel/opt/anaconda3/envs/pytorch/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:107: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name     | Type       | Params
----------------------------------------
0 | network  | Sequential | 816 K 
1 | accuracy | Accuracy   | 0     
----------------------------------------
816 K     Trainable params
0         Non-trainable params
816 K     Total params
3.265     Total estimated model params size (MB)
/Users/tbreuel/opt/anaconda3/envs/pytorch/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_work

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.
/Users/tbreuel/opt/anaconda3/envs/pytorch/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:489: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(
/Users/tbreuel/opt/anaconda3/envs/pytorch/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Validation: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         val_acc            0.9763000011444092
        val_loss            0.07927239686250687
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val_loss': 0.07927239686250687, 'val_acc': 0.9763000011444092}]